In [1]:
from pyspark import SparkConf, SparkContext
import os
sc.stop()
conf = SparkConf().setMaster("local").setAppName("term")
sc = SparkContext(conf=conf)


In [2]:
import math

def map3(x):
    output_list = []
    list_ = x.split(',')
    output_list.append((int(list_[1]), (int(list_[0]), float(list_[2]))))
    return output_list

def map4(x):
    output_list = []
    list_ = x.split(',')
    output_list.append((int(list_[0]), (int(list_[1]))))
    return output_list

def map2(x):
    output_list = []
    list_ = x.split(',')
    output_list.append((int(list_[0]), (int(list_[1]), float(list_[2]))))
    return output_list

def sub_mean_map(x):
    sum_ = 0
    l2 = 0
    list_ = []
    for i in x[1]:
        sum_ += i[1] 
    mean = sum_/len(x[1])
    
    for i in x[1]:
        l2 += (i[1]-mean) * (i[1]-mean)
    for i in x[1]:
        list_.append((i[0], (x[0], i[1] - mean, math.sqrt(l2))))
    return list_

In [3]:
data = sc.textFile("/Users/haoweichung/Desktop/清大/Senior/data_mining/term/ml-latest-small/ratings.csv").filter(lambda x: x != 'userId,movieId,rating,timestamp')

In [4]:
sub_mean = data.flatMap(map3).groupByKey().mapValues(list).flatMap(sub_mean_map).groupByKey().mapValues(list) # (user, (movie, rate-mean)) .flatMap(sub_mean_map)

In [5]:
def map1(x):
    list_ = []
    data_list = sorted(x[1])
    for i in range(len(data_list)):
        for j in range(i+1,len(data_list)):
            if data_list[i][2]*data_list[j][2] != 0 and data_list[i][1]*data_list[j][1] != 0:
                list_.append(((data_list[i][0],data_list[j][0]),((data_list[i][1]*data_list[j][1])/(data_list[i][2]*data_list[j][2]))))
            else:
                list_.append(((data_list[i][0],data_list[j][0]),0))
    return list_

def map5(x):
    list_ = []
    list_.append((x[0][0], (x[0][1], x[1])))
    list_.append((x[0][1],(x[0][0], x[1])))
    return(list_)
 

In [6]:
final = sub_mean.flatMap(map1).reduceByKey(lambda x,y : x+y)

In [7]:
check_finals = final.filter(lambda x: x[1] > 0).flatMap(map5).groupByKey().mapValues(list) #(movie, [(movie, sim),(movie, sim),(movie, sim),(movie, )...])

In [8]:
test = check_finals.collect()



In [ ]:
F = final.sortByKey(ascending=True).collect()
print(F)

In [ ]:
fp = open("/Users/haoweichung/Desktop/清大/Senior/data_mining/term/ml-latest-small/output.txt", "a")
count = 0
for i in F:
    fp.write(str(i[0])+ ',' + str(i[1])+'\n')
    count +=1
print(count)
fp.close()

In [9]:
check = data.flatMap(map3).groupByKey().mapValues(list).map(lambda x: (x[0])).collect()

In [36]:

def find_movie(x):
    for i in test:
        if i[0] == x:
            return i
        
        
def fill_the_space(x):
    list_ = []
    dict_1 = dict(x[1][1])
    print("-----checkpoint-----")
    for i in x[1][0]:
        to_cal = []
        check_max = []
        tmp_1= find_movie(i)

        if tmp_1 == None:
            continue
        tmp_= tmp_1[1]
        sum_ = 0
        div = 0
        for k in tmp_:
            to_append = dict_1.get(k[0])
            if to_append!= None:
                check_max.append((k[1],to_append))
        for k in range(10):
            if len(check_max) < 1:
                break
            max_ = -2
            to_find = (0,0)
            for j in check_max:
                if j[0] > max_:
                    to_find = j
                    max_ = j[0]
            check_max.remove(to_find)
            to_cal.append(to_find)
        if len(to_cal) > 0:
            for k in to_cal:
                sum_ += k[0] * k[1]
                div += k[0]
        
        if div != 0:
            list_.append(((x[0],i), sum_/div))
    return list_




    

            

def find_space(x):
    list_ = []
    check_list = list(set(check)-set(x[1]))
    for i in check_list:
        list_.append((x[0], i))
    return list_
    
to_fill = data.flatMap(map4).groupByKey().mapValues(list).flatMap(find_space).groupByKey().mapValues(list) #(User, ([space1, space2, space3, ....]))


In [37]:
U_to_M = data.flatMap(map2).groupByKey().mapValues(list).sortByKey(ascending=True)

In [35]:
#(movie, [(movie, sim),(movie, sim),(movie, sim),(movie, sim)...]) => check_finals
#(User, ([movie_to_pre1, movie_to_pre2, movie_to_pre3, ....])) => to_fill x[1][0]
#(User, ([movie,movie,movie,movie...])) => U_to_M x[1][1]


-----checkpoint-----
[((153, 3), 1.8825572378415847), ((153, 4), 3.015680108797533), ((153, 5), 2.0046421020486096), ((153, 6), 1.3320963696875419), ((153, 7), 2.212074065955777), ((153, 8), 1.6427387925861385), ((153, 9), 2.176749284880099), ((153, 10), 1.3818448238764123), ((153, 11), 1.6227222338171319), ((153, 12), 1.448930820823491), ((153, 13), 1.8444778364124679), ((153, 14), 1.862930870126243), ((153, 15), 2.687579908046907), ((153, 16), 2.5132557738254966), ((153, 17), 2.332702941838174), ((153, 18), 2.0155865190647297), ((153, 19), 1.478456226392216), ((153, 20), 2.823803650245435), ((153, 21), 1.9457065893438763), ((153, 22), 1.6359987240918037), ((153, 23), 3.53082477393811), ((153, 24), 2.0996396460126046), ((153, 25), 2.95467876542415), ((153, 26), 3.0764744038937657), ((153, 27), 1.7857183433429709), ((153, 28), 1.985880131039653), ((153, 29), 2.043207127236661), ((153, 31), 2.0576728604176324), ((153, 32), 2.9858147483660553), ((153, 34), 2.291649461195235), ((153, 30),

In [38]:
final_predict = to_fill.join(U_to_M)
# (User, [(movie_don't_have), (movie_had)])
# print(final_predict.collect())
# print(final_predict.collect()[152])

In [39]:
ff = final_predict.flatMap(fill_the_space).collect()

In [45]:
F = sorted(ff, key = lambda x : x[0])
fp = open("/Users/haoweichung/Desktop/清大/Senior/data_mining/term/ml-latest-small/prediction.txt", "a")
count = 0
for i in F:
    fp.write(str(i[0])+ ',' + str(round(i[1],2))+'\n')
    count +=1
print(count)
fp.close()

3541186
